In [1]:
!pip install jupyter-dash


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install dash_core_components


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install dash_html_components


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Load your dataset
df = pd.read_csv("dataset_part_3.csv")  # replace with your path
max_payload = df['PayloadMass_kg'].max()
min_payload = df['PayloadMass_kg'].min()

# Create Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("SpaceX Launch Dashboard", style={'textAlign': 'center'}),

    dcc.Dropdown(
        id='site_dropdown',
        options=[{'label': 'All Sites', 'value': 'All Sites'}] + 
                [{'label': site, 'value': site} for site in df['Launch_Site'].unique()],
        value='All Sites',
        searchable=True,
        placeholder="Select a Launch Site"
    ),

    html.Br(),
    dcc.Graph(id='success-pie-chart'),

    html.P("Payload Mass Range (kg):"),
    dcc.RangeSlider(
        id='payload_slider',
        min=0, max=10000, step=500,
        value=[min_payload, max_payload],
        marks={i: f"{i} kg" for i in range(0, 10001, 2000)}
    ),

    html.Br(),
    dcc.Graph(id='success-payload-scatter-chart')
])

@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site_dropdown', 'value')
)
def update_pie_chart(site):
    if site == 'All Sites':
        fig = px.pie(df[df['Success'] == 1], names='Launch_Site',
                     title='Total Successful Launches by Site', hole=0.3)
    else:
        site_df = df[df['Launch_Site'] == site]
        fig = px.pie(site_df, names='Success',
                     title=f'Success vs Failure for {site}', hole=0.3)
    return fig

@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site_dropdown', 'value'),
     Input('payload_slider', 'value')]
)
def update_scatter(site, payload_range):
    low, high = payload_range
    filtered_df = df[(df['PayloadMass_kg'] >= low) & (df['PayloadMass_kg'] <= high)]

    if site != 'All Sites':
        filtered_df = filtered_df[filtered_df['Launch_Site'] == site]

    fig = px.scatter(
        filtered_df, x='PayloadMass_kg', y='Success',
        color='BoosterVersion', size='PayloadMass_kg',
        hover_data=['Orbit', 'Outcome'],
        title='Payload vs. Success by Booster Version'
    )
    return fig

# ✅ Standard run method (works both in script and notebook)
if __name__ == '__main__':
    app.run(debug=True)


C:\Users\aliju\AppData\Local\Temp\ipykernel_14052\1384436101.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
